In [1]:
import numpy as np
import os, cv2, math
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def only_face_part_detect(img_vec):
    gray = np.array(img_vec, dtype='uint8')
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 7)
    if len(faces)>0:
        for (x,y,w,h) in faces:
            return img_vec[x:x+w,y:y+h]
    return img_vec

# READING DATA

In [3]:
# Summarizing the pre-processing:
# train, test are arrays containg tupule (i,j) where i is the label of image, eg. Subject01. 
# j in the tupule is flattened image matrix.
# A matrix contains images as rows (M, N**2) dimension. M: number of images N**2: number of pixels
TRAINING_IMAGES_OUT_OF_ELEVEN = 7
LENGTH = 243
WIDTH = 320
PROD = LENGTH*WIDTH
BASE_FOLDER = "data/"
ls = os.listdir(BASE_FOLDER)
d = dict()
#Creating label for each image.
for i in range(1,10):
    d["subject0"+str(i)] = []
for i in range(10, 16):
    d["subject"+str(i)] = []
for i in ls:
    img = Image.open(BASE_FOLDER+i)
    img = np.array(img, dtype = np.uint8)
#     img = cv2.resize(img,(LENGTH, WIDTH))
#     img = only_face_part_detect(img)
#     img = cv2.resize(img,(LENGTH, WIDTH))
    img = img.flatten()
    d[i[:9]].append(img)  

In [4]:
train = []
test = []
for i in d:
    for j in d[i][:TRAINING_IMAGES_OUT_OF_ELEVEN]:
        train.append((i,j))
    for j in d[i][TRAINING_IMAGES_OUT_OF_ELEVEN:]:
        test.append((i,j))
train = np.array(train)
test = np.array(test)
print(test.shape, train.shape)

(60, 2) (105, 2)


In [5]:
A = np.ndarray((len(train), PROD))
for i in range(len(train)):
    A[i] = train[i][1]
# Summarizing the pre-processing:
# train, test are arrays containg tupule (i,j) where i is the label of image, eg. Subject01. 
# j in the tupule is flattened image matrix.
# A matrix contains images as rows (M, N**2) dimension. M: number of images N**2: number of pixels.

# Mean Calculation

In [6]:
mean = np.mean(A, axis=0)
print(mean)
A = A - mean
# A according to the paper consists of coloumns representing images (N**2, M) dimensions
A = np.transpose(A)
print(A.shape)

[123.45714286 123.53333333 123.74285714 ...  68.          68.
  68.        ]
(77760, 105)


In [7]:
# TO SEE MEAN IMAGE uncomment this portion.
mean_arr  = np.resize(mean, (LENGTH, WIDTH))
cv2.imshow("original", np.array(mean_arr, dtype = np.uint8))

cv2.waitKey()
cv2.imwrite("mean.jpg", mean_arr)
cv2.destroyAllWindows()

# Main Algorithm implementation. 

In [8]:
print(A.shape)
L = np.dot(np.transpose(A), A)
print(L.shape)

(77760, 105)
(105, 105)


In [9]:
from numpy import linalg as LA
eigenval, eigenvec = LA.eigh(L)
# eigenvec = eigenvec.tolist()
print("Done:)")

Done:)


## PCA Dimensionality Reduction

In [10]:
# We take those eigenvectors corresponding to 0.85 of total energy of eigenvalues.
w2 = eigenval
w2 = w2/sum(w2)
energy = 0.85
target = []
while(energy>0):
    x = np.argmax(w2)
    energy-=w2[x]
    target.append(x)
    w2[x] = 0
print("Original number of eigenvectors",len(eigenvec))
eigenvec = eigenvec[:, target]
print("After PCA number of eigenvectors",eigenvec.shape[1])

Original number of eigenvectors 105
After PCA number of eigenvectors 16


## A*A.T eigenvectors and normalizing eigenvectors.

In [11]:
# A*A.T eigenvec are denoted by u 
u = np.dot(A, eigenvec)
print(u.shape)
norms = np.linalg.norm(u, axis = 0)
u = u / norms
print(u.shape)

(77760, 16)
(77760, 16)


In [13]:
# Weights are vector representation of each training image on the eigenface plane
weights = np.dot(np.transpose(u), A)
# Projection gives projection of each training image on eigenface plane
projection = np.dot(weights.T, u.T)

In [20]:
# For visualizing projections of 3 images uncomment and run the following code. 
# Press any key to continue on image displayed. 
# for i in [1,14,24]:
#     mean_arr  = np.resize(A[:,i], (LENGTH, WIDTH))
#     cv2.imshow("original_without_mean"+str(i)+".jpg", np.array(mean_arr, dtype = np.uint8))
#     cv2.waitKey()
#     cv2.destroyAllWindows()
#     cv2.imwrite("original_without_mean"+str(i)+".jpg", np.array(mean_arr, dtype = np.uint8))
#     mean_arr  = np.resize(A[:,i]+mean, (LENGTH, WIDTH))
#     cv2.imshow("original", np.array(mean_arr, dtype = np.uint8))
#     cv2.waitKey()
#     cv2.destroyAllWindows()
#     cv2.imwrite("original"+str(i)+".jpg", np.array(mean_arr, dtype = np.uint8))
#     mean_arr  = np.resize(projection[i], (LENGTH, WIDTH))
#     cv2.imshow("projection", np.array(mean_arr, dtype = np.uint8))
#     cv2.waitKey()
#     cv2.destroyAllWindows()
#     cv2.imwrite("prediction"+str(i)+".jpg", np.array(mean_arr, dtype = np.uint8))

# Testing

In [15]:
def predict(test_vector, weights):
    ans_ = 10**20
    index = -1
    for ct,i in enumerate(np.transpose(weights)):
        # Looping over all training images to find the closest match.
        tans_ = np.linalg.norm(i-test_vector)
        if tans_<ans_:
            # If norm distance less than any previous assign index to it
            ans_ = tans_
            index = ct
    return index

In [16]:
#Finding vector representation of all input images.
acc = 0
for test_img in test:
    # Loading label and flattened test image
    label = test_img[0]
    test_img = test_img[1]
    #Subracting mean from test image
    test_img = test_img - mean
    # Finding weights of eigenface space represenatation
    test_vector = np.dot(np.transpose(u), test_img)
    # Calling predict unction to find closest training image to this image.
    prediction = predict(test_vector, weights)
    # If test_label match the predicted label increase acc.
    if train[prediction][0] == label:
        acc += 1
print("Accuracy of predictions: ",round(acc/len(test)*100, 2), "%")
    

    

Accuracy of predictions:  86.67 %
